In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Nov 17 01:14:06 2022

@author: vkedu
"""
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.preprocessing import LabelEncoder
import pickle
from sklearn.model_selection import cross_val_score, train_test_split

In [ ]:
df = pd.read_csv("Data/autos.csv", header=0, sep=',', encoding='Latin1',)
df.head()

In [ ]:
df[df.seller != 'gewerblich']
df=df.drop('seller',1)

In [ ]:
df[df.offerType != 'Gesuch']
df = df.drop('offerType',1)

In [ ]:
df=df[(df.powerPS>50)&(df.powerPS<900)]
df=df[(df.yearOfRegistration >= 1950)&(df.yearOfRegistration<2017)]
df.drop(['name','abtest','dateCrawled','nrOfPictures','lastSeen','postalCode','dateCreated'], axis='columns',inplace=True)

In [ ]:
new_df=df.copy()
new_df=new_df.drop_duplicates(['price','vehicleType','yearOfRegistration','gearbox','powerPS','model','kilometer','monthOfRegistration','fuelType','notRepairedDamage'])

In [ ]:
new_df.gearbox.replace(('manuell','automatik'),('manual','automatic'), inplace=True)
new_df.fuelType.replace(('benzin','andere','elecktro'),('petrol','others','electic'), inplace=True)
new_df.vehicleType.replace(('kleinwagen','cabrio','kombi','andere'),('small car','convertible','combination','others'), inplace=True)
new_df.notRepairedDamage.replace(('ja','nein'),('Yes','No'), inplace=True)


In [ ]:
new_df=new_df[(new_df.price>=100)&(new_df.price<=150000)]

In [ ]:
new_df['notRepairedDamage'].fillna(value='not-declared',inplace=True)
new_df['fuelType'].fillna(value='not-declared',inplace=True)
new_df['gearbox'].fillna(value='not-declared',inplace=True)
new_df['vehicleType'].fillna(value='not-declared',inplace=True)
new_df['model'].fillna(value='not-declared',inplace=True)

In [ ]:
new_df.to_csv("autos_preprocessed.csv")

In [ ]:
l=['gearbox','notRepairedDamage','fuelType','vehicleType','model','brand']
m={}
for i in l:
    m[i]=LabelEncoder()
    m[i].fit(new_df[i])
    tr=m[i].transform(new_df[i])
    np.save(str('classes'+i+'.npy'),m[i].classes_)
    new_df.loc[:,i+'_labels']=pd.Series(tr,index=new_df.index)

In [ ]:
l2=new_df[['price','yearOfRegistration','powerPS','kilometer','monthOfRegistration'] + [x+"_labels" for x in l]]

In [ ]:
Y=l2.iloc[:,0].values
X=l2.iloc[:,1:].values

In [ ]:
Y=Y.reshape(-1,1)

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y, test_size=0.3, random_state=3)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

In [ ]:
r=RandomForestRegressor(n_estimators=1000,max_depth=10,random_state=34)

In [ ]:
r.fit(X_train,np.ravel(Y_train,order='C'))
y_pred=r.predict(X_test)
y_pred

In [ ]:
fn="resale_model.sav"
pickle.dump(r,open(fn,'wb'))